In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from decoders import DenseNN

import metrics

from sklearn import datasets
from sklearn.model_selection import train_test_split


# torch.manual_seed(123) ##For reproducibility. This will make sure that same random weights are initialized each time.


In [2]:
## input data

## output data
X, Y = datasets.load_boston(return_X_y=True)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.9)
# Y_train = Y_train.reshape(-1,1)
# Y_test = Y_test.reshape(-1,1)
# print(type(Y_train))
# print(Y_test.shape)

X_train, X_test, Y_train, Y_test = torch.tensor(X_train, dtype=torch.float32),\
                                   torch.tensor(X_test, dtype=torch.float32),\
                                   torch.tensor(Y_train, dtype=torch.float32),\
                                   torch.tensor(Y_test, dtype=torch.float32)

samples, in_features = X_train.shape

_ , out_features = Y_train.view(Y_train.shape[0],1).shape

/Users/Munib/opt/anaconda3/envs/torchenv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Ca

In [3]:
if 'net' in locals():
    del net

## dense neural network
units = [10,40,5] # length of units determines number of hidden layers, each with units[i] units
# units = [200,40,5] # length of units determines number of hidden layers, each with units[i] units
dropout = 0 # between [0,1] - probability of dropping parameters between each fully connected hidden layer

net = DenseNN(in_features, out_features, units, dropout=dropout) # in_features=nNeurons, out_features=nMoveFeatures
print(net)

DenseNN(
  (act): ReLU()
  (fc1): Linear(in_features=13, out_features=10, bias=True)
  (hidden): ModuleList(
    (0): Linear(in_features=10, out_features=40, bias=True)
    (1): Linear(in_features=40, out_features=5, bias=True)
  )
  (out): Linear(in_features=5, out_features=1, bias=True)
)


In [4]:
## train network
loss_func = nn.MSELoss()
learning_rate = torch.tensor(1/1e3) # 0.001

# optimizer = torch.optim.SGD(params=net.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate)

epochs = 10000

net.fit(loss_func, optimizer, X_train, Y_train, epochs)

Epoch: 0  ||  MSE: 745.1602172851562
Epoch: 500  ||  MSE: 24.798704147338867
Epoch: 1000  ||  MSE: 14.169316291809082
Epoch: 1500  ||  MSE: 12.486102104187012
Epoch: 2000  ||  MSE: 11.536774635314941
Epoch: 2500  ||  MSE: 10.698667526245117
Epoch: 3000  ||  MSE: 9.858511924743652
Epoch: 3500  ||  MSE: 8.9229097366333
Epoch: 4000  ||  MSE: 8.221458435058594
Epoch: 4500  ||  MSE: 7.82087516784668
Epoch: 5000  ||  MSE: 7.181705951690674
Epoch: 5500  ||  MSE: 6.736398220062256
Epoch: 6000  ||  MSE: 6.445782661437988
Epoch: 6500  ||  MSE: 6.241292953491211
Epoch: 7000  ||  MSE: 6.017441749572754
Epoch: 7500  ||  MSE: 5.854900360107422
Epoch: 8000  ||  MSE: 6.077504634857178
Epoch: 8500  ||  MSE: 5.628582000732422
Epoch: 9000  ||  MSE: 5.615628242492676
Epoch: 9500  ||  MSE: 5.572151184082031


In [5]:
# predict on training set

Y_pred = net.predict(X_test) ## Make Predictions on test dataset

# R^2 between predictions and ground truth
r2=metrics.get_R2(Y_test.view(Y_test.shape[0],1),Y_pred)
print('R^2:', r2)

R^2: [[0.92566788]]
